In [1]:
import os
import pandas as pd
import pickle
import ast
import sys
import re
import numpy as np

In [2]:
OUT_DIR="/media/SSD/paper_data/"

In [3]:
sensor_tag="all"
seeds = [0]
preprocessings = ["none"]
models = ["resnet"]
hierarchicals = ['top', 'arith', 'logic', 'compare', 'shift', 'load', 'store', 'branch', 'jump']
datasets = ["OUT1/random"]
i = 10000
topks = [2, 3, 4, 5, 6]
topk_empty = []

columns = ['dataset', 'model', 'hierarchical', 'preprocessing', 'seed', 'kfold', 'accuracy']
accuracies = pd.DataFrame(columns=columns)

jobs_to_repeat = []

results_idx = 0

for seed in seeds:
    for preprocessing in preprocessings:
        for model in models:
            for dataset in datasets:
                for hierarchical in hierarchicals:
                
                    model_small=model.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                    dataset_small=dataset.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                    preprocessing_small=preprocessing.replace("_", "-").replace("+", "-").replace("/", "-").lower()

                    acc_file=OUT_DIR+"/sakura/Exp-"+dataset+"/results/"+model+"_h"+hierarchical+"_p"+preprocessing+"_"+str(i)+"_s"+str(seed)+"-sensors-"+sensor_tag+"_10fold/out/kfold_results.txt"

                    # Check if job was completed
                    if(os.path.exists(acc_file)!=True):
                        print("Results for "+"Exp-"+dataset+"/results/"+model+"_h"+hierarchical+"_p"+preprocessing+"_"+str(i)+"_s"+str(seed)+"-sensors-"+sensor_tag+"_10fold"+" not yet ready!")
                        jobs_to_repeat.append("prs-"+dataset_small+"-"+model_small+"_h"+hierarchical+"-p"+preprocessing_small+"-"+str(i)+"-s"+str(seed)+"-sens-"+sensor_tag)
                        for kfold_tmp in range(10):
                            accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, hierarchical, preprocessing, 0, kfold_tmp, "-"]
                            results_idx = results_idx+1
                        accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, hierarchical, preprocessing, 0, "avg", "-"]
                        results_idx = results_idx+1
                        continue

                    # Check if data was stored safetly
                    num_lines = sum(1 for line in open(acc_file))
                    if(num_lines <= 11):
                        print("Missing data for "+"Exp-"+dataset+"/results/"+model+"_h"+hierarchical+"_p"+preprocessing+"_"+str(i)+"_s"+str(seed)+"-sensors-"+sensor_tag+"_10fold")
                        jobs_to_repeat.append("prs-"+dataset_small+"-"+model_small+"_h"+hierarchical+"-p"+preprocessing_small+"-"+str(i)+"-s"+str(seed)+"-sens-"+sensor_tag)
                        for kfold_tmp in range(10):
                            accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, hierarchical, preprocessing, 0, kfold_tmp, "-"]
                            results_idx = results_idx+1
                        accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, hierarchical, preprocessing, 0, "avg", "-"]
                        results_idx = results_idx+1
                        continue
                    #print(out_path)

                    file1 = open(acc_file, 'r')
                    Lines = file1.readlines()

                    for count, line in enumerate(Lines):
                        #print("Line{}: {}".format(count, line.strip()))
                        if(count>=1 and count <=10):
                            acc_evol=OUT_DIR+"/sakura/Exp-"+dataset+"/results/"+model+"_h"+hierarchical+"_p"+preprocessing+"_"+str(i)+"_s"+str(seed)+"-sensors-"+sensor_tag+"_10fold/out/log/LSTM+CNN_kf"+str(count-1)+".txt"
                            kf_accuracy = re.findall("\d+\.\d+", line.strip())[1]
                            #row = pd.DataFrame(['Exp-IN', 'LSTM+CNN', 'none', 0, count, kf_accuracy])
                            #accuracies = pd.concat([accuracies, row], axis=0, ignore_index=True)
                            accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, hierarchical, preprocessing, 0, count-1, kf_accuracy]
                            results_idx = results_idx+1
                        if(count==12):
                            kf_accuracy = re.findall("\d+\.\d+", line.strip())[0]
                            accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, hierarchical, preprocessing, 0, "avg", kf_accuracy]
                            results_idx = results_idx+1

                    file1.close()

In [14]:
accuracies.to_csv(OUT_DIR+"/sakura/accuracies/accuracies_hierarchical_all.csv", index=False)
accuracies.iloc[:, 0:7].loc[accuracies.kfold=='avg'].reset_index(drop=True).to_csv(OUT_DIR+"/sakura/accuracies/accuracies_hierarchical_averaged.csv", index=False)